In [1]:
%matplotlib inline
import sys

import arviz as az
import numpy as np
import pymc as pm

if not sys.warnoptions:
    import warnings
warnings.simplefilter("ignore")

from forest import *

Loading BokehJS ...

## Test 1

### Create Models

In [2]:
dict_cmp = {}
d1 = az.load_arviz_data("centered_eight")
d2 = az.load_arviz_data("non_centered_eight")
dict_cmp["mA"] = d1
dict_cmp["mB"] = d2

### Build Dashboard

In [3]:
forest_dashboard = ForestDashboard(dict_cmp)
forest_dashboard.dashboard_forest()

Column
    [0] Row
        [0] MultiSelect(name='ModelSelect', options=['mA', 'mB'], value=['mA'])
    [1] FloatSlider(name='HDI Probability', step=0.05, value=0.7, width=200)
    [2] Row
        [0] Select(name='Model', options=OrderedDict([('mA', ...]), value='mA')
        [1] Select(name='Data variable', options=OrderedDict([('mu', ...]), value='mu')
        [2] Select(name='Coor variable')
    [3] Tabs
        [0] Row
            [0] ParamFunction(function)
        [1] Column
            [0] Row
                [0] Checkbox(name='Ridgeplot Truncate')
            [1] Row
                [0] RangeSlider(name='Ridgeplot Quantiles', step=0.01, value=(0.25, 0.75), value_end=0.75, value_start=0.25, width=200)
                [1] FloatSlider(name='Ridgeplot Overlap', step=0.05, value=0.7, width=200)
            [2] ParamFunction(function)